In [6]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

os.chdir("../")
load_dotenv()

OPENAI_SECRET_KEY = os.getenv("OPENAI_SECRET_KEY")

Make a custom tool which writes an entry to a diary.
<br>Reference: https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/

In [2]:
import datetime
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [3]:
class DiaryEntry:

    def __init__(self, date: datetime.date, text: str):
        self.date = date
        self.text = text

    def __str__(self):
        return self.text

class Diary:

    def __init__(self):

        self.entries = {} # {datetime: [entry, ..]}

    def write_entry(self, entry):

        self.entries[entry.date] = self.entries.get(entry.date, []) + [entry]

    def get_entries(self, date):

        return self.entries.get(date, [])

In [4]:
test_diary = Diary()
test_diary.write_entry(
    DiaryEntry(date = datetime.date(2024,6,6), text = "I ate a lot")
)
print(test_diary.get_entries(datetime.date(2024,6,6))[0])

I ate a lot


In [7]:
class WriteToDiaryInput(BaseModel):
    date: datetime.date = Field(description = "The date in the diary wherein the text will be written. Defaults to today if not provided.")
    text: str = Field(description = "The text to be written into the diary")
    
DIARY = Diary()

@tool("write-to-diary-tool", args_schema = WriteToDiaryInput, return_direct = True)
def write_to_diary(text: str, date: datetime.date = datetime.date.today()):
    """
    Writes the text into the diary in the page of the corresponding date.
    Writes to the page for today if date is not provided.
    """
    DIARY.write_entry(
        DiaryEntry(
            date = date,
            text = text
        )
    )
    return True

Make a conversational agent that can use the tool to write to the diary when commanded